In [2]:
# LangChain 接入Yuan模型
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='./model/autodl-fs')

d:\Anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 调用模型
from LLM import Yuan2_LLM
llm = Yuan2_LLM('model/autodl-fs/IEITYuan/Yuan2-2B-Mars-hf')
print(llm("你是谁"))

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Creat tokenizer...
Creat model...


d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 我是一个人工智能助手，专门设计来帮助您解决问题和提供信息。无论您有什么需要，只要告诉我，我就会尽力帮助您。<eod>


In [4]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.pdf import PyMuPDFLoader
# 初始化一个空列表来存储 PDF 文件路径
pdf_file_paths = []

# 指定文件夹路径
folder_path = './data'

# 使用 os.walk 遍历文件夹及其子文件夹
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # 检查文件是否为 PDF 文件
        if file.endswith('.pdf'):
            # 构建完整的文件路径并添加到列表中
            file_path = os.path.join(root, file)
            pdf_file_paths.append(file_path)

# 打印前三个 PDF 文件的路径
print(pdf_file_paths[:3])
print(len(pdf_file_paths))

[]
0


In [5]:
# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in pdf_file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))

len(loaders)

0

In [8]:
# 下载文件并存储到text
texts = []
for loader in loaders: texts.extend(loader.load())
len(texts)

3462

In [ ]:
text = texts[0]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content}", 
    sep="\n------\n")

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 文档分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,length_function=len)
split_docs = text_splitter.split_documents(texts)
print(len(split_docs))

6533


In [22]:
from sentence_transformers import SentenceTransformer

# 指定向量模型名称
model_name = "moka-ai/m3e-base"

# 创建模型实例并将模型保存到指定路径
model = SentenceTransformer(model_name)
model.save('\\model\\m3e-base')

In [8]:
# 向量化
from transformers.utils import is_torch_cuda_available, is_torch_mps_available
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# 词嵌入模型
EMBEDDING_DEVICE = "cuda" if is_torch_cuda_available() else "mps" if is_torch_mps_available() else "cpu"
embeddings_model = HuggingFaceEmbeddings(model_name='./model/m3e-base', model_kwargs={'device': 'cuda'})

d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [ ]:
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings_model)

In [24]:
# 指定保存的路径
save_path = "./faiss_index"

# 保存向量数据库
vectorstore.save_local(save_path)

In [10]:
from langchain.vectorstores import FAISS

# 指定保存的路径
save_path = "./faiss_index"

# 加载向量数据库
vectorstore = FAISS.load_local(save_path, embeddings_model, allow_dangerous_deserialization=True)

In [11]:
print(vectorstore.index)
print(vectorstore.index.ntotal)  # 向量总数
print(vectorstore.index.d)       # 向量维度

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000012FF3F42810> >
6533
768


In [ ]:
# 查看元数据
metadata_list = vectorstore.docstore._dict.values()
for metadata in metadata_list:
    print(metadata)

In [27]:
# 向量查询
query = '东方健康膳食模式'
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

8 
 
1.得舒饮食（DASH）。富含新鲜蔬菜、水果、低脂（或
脱脂）乳制品、禽肉、鱼、大豆和坚果以及全谷物，限制含
糖饮料和红肉的摄入，
饱和脂肪酸和胆固醇水平低，
富含钾、
镁、钙等矿物质、优质蛋白质和膳食纤维。 
2.东方健康膳食模式。我国东南沿海地区居民高血压、
脑卒中的风险较低，期望寿命也较高，东南沿海一带的代表
性膳食统称为东方健康膳食模式。其主要特点是清淡少盐、
食物多样、谷物为主，蔬菜水果充足，鱼虾等水产品丰富，
奶类、豆类丰富等，并且具有较高的身体活动量。 
3.中国心脏健康膳食（CHH-Diet）。与中国城市人群普
通膳食相比，
本膳食模式减少钠摄入，
同时减少了脂肪摄入，
增加了蛋白质、碳水化合物、钾、镁、钙和膳食纤维摄入。


In [28]:
#向量检索
retriever = vectorstore.as_retriever()
docs = retriever.get_relevant_documents("东方健康膳食模式")
print(docs[0].page_content)

8 
 
1.得舒饮食（DASH）。富含新鲜蔬菜、水果、低脂（或
脱脂）乳制品、禽肉、鱼、大豆和坚果以及全谷物，限制含
糖饮料和红肉的摄入，
饱和脂肪酸和胆固醇水平低，
富含钾、
镁、钙等矿物质、优质蛋白质和膳食纤维。 
2.东方健康膳食模式。我国东南沿海地区居民高血压、
脑卒中的风险较低，期望寿命也较高，东南沿海一带的代表
性膳食统称为东方健康膳食模式。其主要特点是清淡少盐、
食物多样、谷物为主，蔬菜水果充足，鱼虾等水产品丰富，
奶类、豆类丰富等，并且具有较高的身体活动量。 
3.中国心脏健康膳食（CHH-Diet）。与中国城市人群普
通膳食相比，
本膳食模式减少钠摄入，
同时减少了脂肪摄入，
增加了蛋白质、碳水化合物、钾、镁、钙和膳食纤维摄入。


/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_template = "你是一名资深的医生，对于高血压有很深的经验"
human_template = "请根据以下文档回答我的问题：\n\n{docs}\n\n问题：{question}"

prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human" , human_template)
    ])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
question = "高血压患者应该怎么饮食？"
docs = retriever.get_relevant_documents(question)

chain.invoke({"question": question, "docs": docs})

' 高血压患者应该遵循低钠、低脂、高纤维的原则，合理膳食，科学食养。具体来说，应减少钠盐摄入量，增加富钾食物摄入，控制脂肪摄入，保持适量的热量摄入，并多食用富含维生素、矿物质和纤维素的食物。同时，应该避免高钠盐、高脂肪、高胆固醇和高糖分的食物，戒烟限酒，保持情绪稳定，并根据个体情况制定个性化的饮食计划。<eod>'